# Install SAR, collect data, plot results

In [ ]:
import pandas
import json
import datetime
from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates

To install SAR on CentOS 7,
```bash
sudo yum install sysstat
```

Once installed you can launch the service that writes to `/var/log/sa` using
```bash
sudo systemctl start sysstat
```

Then
```bash
sadf -T -j /var/log/sa/sa* -- -A > file.json
```
produces the file consumed below.

In [ ]:
column_meanings={
"tps": "Total number of transfers per second that were issued to physical devices. A transfer is an I/O request to a physical device. Multiple logical requests can be combined into a single I/O request to the device. A transfer is of indeterminate size.",
"rtps": "Total number of read requests per second issued to physical devices.",
"wtps": "Total number of write requests per second issued to physical devices.",
"bread/s": "Total amount of data read from the devices in blocks per second. Blocks are equivalent to sectors with 2.4 kernels and newer and therefore have a size of 512 bytes. With older kernels, a block is of indeterminate size.",
"bwrtn/s": "Total amount of data written to devices in blocks per second.",
"pgpgin/s": "Total number of kilobytes the system paged in from disk per second. Note: With old kernels (2.2.x) this value is a number of blocks per second (and not kilobytes).",
"pgpgout/s": "Total number of kilobytes the system paged out to disk per second. Note: With old kernels (2.2.x) this value is a number of blocks per second (and not kilobytes).",
"fault/s": "Number of page faults (major + minor) made by the system per second (post 2.5 kernels only). This is not a count of page faults that generate I/O, because some page faults can be resolved without I/O.",
"majflt/s": "Number of major faults the system has made per second, those which have required loading a memory page from disk (post 2.5 kernels only).",
"proc/s": "Total number of processes created per second.",
"tps": "Indicate the number of transfers per second that were issued to the device. Multiple logical requests can be combined into a single I/O request to the device. A transfer is of indeterminate size.",
"rd_sec/s": "Number of sectors read from the device. The size of a sector is 512 bytes.",
"wr_sec/s": "Number of sectors written to the device. The size of a sector is 512 bytes.",
"avgrq-sz": "The average size (in sectors) of the requests that were issued to the device.",
"avgqu-sz": "The average queue length of the requests that were issued to the device.",
"await": "The average time (in milliseconds) for I/O requests issued to the device to be served. This includes the time spent by the requests in queue and the time spent servicing them.",
"svctm": "The average service time (in milliseconds) for I/O requests that were issued to the device.",
"%util": "Percentage of CPU time during which I/O requests were issued to the device (bandwidth utilization for the device). Device saturation occurs when this value is close to 100%.",
"IFACE": "Name of the network interface for which statistics are reported.",
"rxpck/s": "Total number of packets received per second.",
"txpck/s": "Total number of packets transmitted per second.",
"rxbyt/s": "Total number of bytes received per second.",
"txbyt/s": "Total number of bytes transmitted per second.",
"rxcmp/s": "Number of compressed packets received per second (for cslip etc.).",
"txcmp/s": "Number of compressed packets transmitted per second.",
"rxmcst/s": "Number of multicast packets received per second.",
"IFACE": "Name of the network interface for which statistics are reported.",
"rxerr/s": "Total number of bad packets received per second.",
"txerr/s": "Total number of errors that happened per second while transmitting packets.",
"coll/s": "Number of collisions that happened per second while transmitting packets.",
"rxdrop/s": "Number of received packets dropped per second because of a lack of space in linux buffers.",
"txdrop/s": "Number of transmitted packets dropped per second because of a lack of space in linux buffers.",
"txcarr/s": "Number of carrier-errors that happened per second while transmitting packets.",
"rxfram/s": "Number of frame alignment errors that happened per second on received packets.",
"rxfifo/s": "Number of FIFO overrun errors that happened per second on received packets.",
"txfifo/s": "Number of FIFO overrun errors that happened per second on transmitted packets.",
"call/s": "Number of RPC requests made per second.",
"retrans/s": "Number of RPC requests per second, those which needed to be retransmitted (for example because of a server timeout).",
"read/s": "Number of 'read' RPC calls made per second.",
"write/s": "Number of 'write' RPC calls made per second.",
"access/s": "Number of 'access' RPC calls made per second.",
"getatt/s": "Number of 'getattr' RPC calls made per second.",
"scall/s": "Number of RPC requests received per second.",
"badcall/s": "Number of bad RPC requests received per second, those whose processing generated an error.",
"packet/s": "Number of network packets received per second.",
"udp/s": "Number of UDP packets received per second.",
"tcp/s": "Number of TCP packets received per second.",
"hit/s": "Number of reply cache hits per second.",
"miss/s": "Number of reply cache misses per second.",
"sread/s": "Number of 'read' RPC calls received per second.",
"swrite/s": "Number of 'write' RPC calls received per second.",
"saccess/s": "Number of 'access' RPC calls received per second.",
"sgetatt/s": "Number of 'getattr' RPC calls received per second.",

"totsck": "Total number of used sockets.",
"tcpsck": "Number of TCP sockets currently in use.",
"udpsck": "Number of UDP sockets currently in use.",
"rawsck": "Number of RAW sockets currently in use.",
"ip-frag": "Number of IP fragments currently in use.",

"runq-sz": "Run queue length (number of processes waiting for run time).",
"plist-sz": "Number of processes and threads in the process list.",
"ldavg-1": "System load average for the last minute.",
"ldavg-5": "System load average for the past 5 minutes.",
"ldavg-15": "System load average for the past 15 minutes.",

"kbmemfree": "Amount of free memory available in kilobytes.",
"kbmemused": "Amount of used memory in kilobytes. This does not take into account memory used by the kernel itself.",
"%memused": "Percentage of used memory.",
"kbbuffers": "Amount of memory used as buffers by the kernel in kilobytes.",
"kbcached": "Amount of memory used to cache data by the kernel in kilobytes.",
"kbswpfree": "Amount of free swap space in kilobytes.",
"kbswpused": "Amount of used swap space in kilobytes.",
"%swpused": "Percentage of used swap space.",
"kbswpcad": "Amount of cached swap memory in kilobytes. This is memory that once was swapped out, is swapped back in but still also is in the swap area (if memory is needed it doesn't need to be swapped out again because it is already in the swap area. This saves I/O).",

"frmpg/s": "Number of memory pages freed by the system per second. A negative value represents a number of pages allocated by the system. Note that a page has a size of 4 kB or 8 kB according to the machine architecture.",
"bufpg/s": "Number of additional memory pages used as buffers by the system per second. A negative value means fewer pages used as buffers by the system.",
"campg/s": "Number of additional memory pages cached by the system per second. A negative value means fewer pages in the cache.",

"%user": "Percentage of CPU utilization that occurred while executing at the user level (application).",
"%nice": "Percentage of CPU utilization that occurred while executing at the user level with nice priority.",
"%system": "Percentage of CPU utilization that occurred while executing at the system level (kernel).",
"%iowait": "Percentage of time that the CPU or CPUs were idle during which the system had an outstanding disk I/O request.",
"%steal": "Show the percentage of time spent in involuntary wait by the virtual CPU or CPUs while the hypervisor was servicing another virtual processor.",
"%idle": "Percentage of time that the CPU or CPUs were idle and the system did not have an outstanding disk I/O request. Note: In virtualized hardware environment, this also includes the time spent in involuntary wait by the virtual CPU or CPUs while the hypervisor was servicing another virtual processor.",

"dentunusd": "Number of unused cache entries in the directory cache.",
"file-sz": "Number of used file handles.",
"inode-sz": "Number of used inode handlers.",
"super-sz": "Number of super block handlers allocated by the kernel.",
"%super-sz": "Percentage of allocated super block handlers with regard to the maximum number of super block handlers that Linux can allocate.",
"dquot-sz": "Number of allocated disk quota entries.",
"%dquot-sz": "Percentage of allocated disk quota entries with regard to the maximum number of cached disk quota entries that can be allocated.",
"rtsig-sz": "Number of queued RT signals.",
"%rtsig-sz": "Percentage of queued RT signals with regard to the maximum number of RT signals that can be queued.",

"cswch/s": "Total number of context switches per second.",

"pswpin/s": "Total number of swap pages the system brought in per second.",
"pswpout/s": "Total number of swap pages the system brought out per second.",

"minflt/s": "Total number of minor faults the process has made per second, those which have not required loading a memory page from disk.",
"majflt/s": "Total number of major faults the process has made per second, those which have required loading a memory page from disk.",
"%user": "Percentage of CPU used by the process while executing at the user level (application), with or without nice priority.",
"%system": "Percentage of CPU used by the process while executing at the system level (kernel).",
"nswap/s": "Number of pages from the process address space the system has swapped out per second. This value is always zero with post 2.5 kernels.",
"CPU": "Processor number to which the process is attached.",

"cminflt/s": "Total number of minor faults the child processes have made per second, those which have not required loading a memory page from disk.",
"cmajflt/s": "Total number of major faults the child processes have made per second, those which have required loading a memory page from disk.",
"%cuser": "Percentage of CPU used by the child processes while executing at the user level (application), with or without nice priority.",
"%csystem": "Percentage of CPU used by the child processes while executing at the system level (kernel).",
"cnswap/s": "Number of pages from the child processes address spaces the system has swapped out per second. This value is always zero with post 2.5 kernels.",

"rcvin/s": "Number of receive interrupts per second for current serial line. Serial line number is given in the TTY column.",
"xmtin/s": "Number of transmit interrupts per second for current serial line.",
"framerr/s": "Number of frame errors per second for current serial line.",
"prtyerr/s": "Number of parity errors per second for current serial line.",
"brk/s": "Number of breaks per second for current serial line.",
"ovrun/s": "Number of overrun errors per second for current serial line."
}

In [ ]:
with open("file_2022-08-20_2.json","r") as file_handle:
    file=json.load(file_handle)

In [ ]:
len(file['sysstat']['hosts'])

In [ ]:
file['sysstat']['hosts'][0].keys()

In [ ]:
file['sysstat']['hosts'][0]['restarts']

In [ ]:
len(file['sysstat']['hosts'][0]['statistics'])

In [ ]:
def expand_list(parent_key: str, list_to_expand: list):
    for index, item in enumerate(list_to_expand):
        #print(type(item))
        type_test(parent_key+"__"+str(index), item)

def expand_dict(parent_key: str, dict_to_expand: dict):
    for key, val in dict_to_expand.items():
        #print(type(val))
        type_test(parent_key+"__"+key, val)

def type_test(key: str, to_test):
    global results_dict
    if type(to_test)==str:
        results_dict[key]= to_test
    elif type(to_test)==int:
        results_dict[key]= to_test
    elif type(to_test)==float:
        results_dict[key]= to_test
    elif type(to_test)==dict:
        expand_dict(key, to_test)
    elif type(to_test)==list:
        expand_list(key, to_test)
    else:
        raise Exception("unrecognized type")
    return results_dict
        

results_dict = {}
for key_0, val_0 in file['sysstat']['hosts'][0]['statistics'][0].items():
    #print(key_0)
    if type(val_0)==list:
        expand_list(key_0, val_0)
    elif type(val_0)==dict:
        expand_dict(key_0, val_0)
    else:
        raise Exception("unrecognized type")

In [ ]:
list_of_result_dicts = []
for entry in file['sysstat']['hosts'][0]['statistics']:
    results_dict = {}
    for key_0, val_0 in entry.items():
        if type(val_0)==list:
            expand_list(key_0, val_0)
        elif type(val_0)==dict:
            expand_dict(key_0, val_0)
        else:
            raise Exception("unrecognized type")
    list_of_result_dicts.append(results_dict)

In [ ]:
df = pandas.DataFrame(list_of_result_dicts)

In [ ]:
# https://stackoverflow.com/a/39641223/1164295
df['YYYY-MM-DD H:M:S'] = pandas.to_datetime(df['timestamp__date'] + " " + 
                                        df['timestamp__time'], 
                                        format='%Y-%m-%d %H:%M:%S')

In [ ]:
def create_plot(df, column_name):
    plot_name = column_name.split("__")[-1]
    print("variable from JSON:", plot_name)
    
    for key,val in column_meanings.items():
        if plot_name in key.strip("/s").strip("%").lower():
            print("  column name in man page:", key)
            print("  definition:", val)
    
    if df[column_name].nunique()==1:
        print("  is hidden because all values are same, specifically", 
              df[column_name][0],"\n\n")
    else:
        plt.plot_date(x=df['YYYY-MM-DD H:M:S'], y=df[column_name])

        ax = plt.gca()

        plt.gca().set_title(column_name)
        plt.gca().set_ylabel(column_name)
        plt.gca().set_xlabel("local date time")

        # plt.gcf().autofmt_xdate

        date_format = mpl_dates.DateFormatter('%Y-%m-%d %H:%M:%S')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.xticks(rotation = 85)
        plt.show()

In [ ]:
for column_name in df.columns:
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html
    if ((df[column_name].dtype=="int64") or 
        (df[column_name].dtype=="float64")):
        create_plot(df, column_name)
